## Part 1: Importing Libraries

In [1]:
from src.modules import data_preprocessing as dp
from src.modules import data_visualization as dv
from src.modules import modeling as md

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

## Part 2: Download Dataset

In [2]:
# Declare the ticker symbol. Here we use 5 stocks "AAPL MSFT AMD GOOG META" as an example.
#tickers = ['aapl', 'amd', 'msft', 'meta', 'goog']

In [2]:
tickers = ['amd']

In [9]:
for ticker in tickers:
    # Download the data from Yahoo Finance
    df = dp.download_data(ticker)

    # clean the data
    df = dp.clean_data(df)

    # split the data into training set and testing set
    train_df, test_df = dp.split_data(df)

    # scale the data
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train_df)
    test_scaled = scaler.transform(test_df)

    # create the training set and testing set
    X_train, y_train = dp.create_dataset(train_scaled, 60)
    X_test, y_test = dp.create_dataset(test_scaled, 60)

    # reshape the data
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

    # build the model
    model = md.create_model(X_train=X_train, y_train=y_train)

    # train the model
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

    # save the model
    model.save(f'models/{ticker}_model.h5')


[*********************100%%**********************]  1 of 1 completed
Epoch 1/10
62/62 [==============================] - 21s 136ms/step - loss: 0.0041 - root_mean_squared_error: 0.0641
Epoch 2/10
62/62 [==============================] - 8s 128ms/step - loss: 8.6348e-04 - root_mean_squared_error: 0.0294
Epoch 3/10
62/62 [==============================] - 7s 119ms/step - loss: 8.8042e-04 - root_mean_squared_error: 0.0297
Epoch 4/10
62/62 [==============================] - 8s 128ms/step - loss: 7.5860e-04 - root_mean_squared_error: 0.0275
Epoch 5/10
62/62 [==============================] - 8s 129ms/step - loss: 7.4178e-04 - root_mean_squared_error: 0.0272
Epoch 6/10
62/62 [==============================] - 7s 119ms/step - loss: 8.2919e-04 - root_mean_squared_error: 0.0288
Epoch 7/10
62/62 [==============================] - 7s 116ms/step - loss: 7.2525e-04 - root_mean_squared_error: 0.0269
Epoch 8/10
62/62 [==============================] - 8s 130ms/step - loss: 6.8517e-04 - root_mean_squa

c:\Users\thong.lam\AppData\Local\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Part 4: Data Transformation

In [10]:
print(X_test.shape)

(503, 60, 1)


In [11]:
print(X_train.shape)

(1954, 60, 1)


In [12]:
print(df.shape)

(2517, 1)


In [13]:
# Create the testing data set.
# The testing data set contains the remaining 20% of the data but we have to
# # include the last 60 days from the training data set to predict the first stock price of the test data set
# scaled_data_test = scaled_dataset[num_training_points - window_size:, :]

# # # Create the data sets x_test and y_test
# X_test = []
# y_test = dataset[num_training_points:, :]

# for i in range(60, len(scaled_data_test)):
#     X_test.append(scaled_data_test[i-window_size:i, 0])

# # Convert the values into arrays for easier computation 
# X_test = np.array(X_test)
# X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
# scaled_data_test = scaled_data[num_training_points - window_size:, :]
#X_test, y_test = convert_to_supervised(scaled_data_test, window_size)

# predict the testing data
predictions = model.predict(X_test)
# predictions = scaler.inverse_transform(predictions)
# y_test = scaler.inverse_transform(y_test)
 
# evaluate result by mse and rmse metrics
mse = np.mean(((predictions - y_test) ** 2))
print("MSE", mse)
print("RMSE", np.sqrt(mse))

16/16 [==============================] - 4s 70ms/step
MSE 0.07070305463294783
RMSE 0.26590046000890605
